In [451]:
import numpy as np
import heapq
import random
THINK = 4
REQ = 5
IDLE = 6
BUSY = 7
ARR = 1
DEP = 2
SWCH = 3
TMT = 8
Q_LENGTH = 100
QUANTUM = 0.020
SERVICE_TIME = 0.048
THINK_TIME = 6
TIMEOUT = 10
np.random.seed(42)

In [452]:
class Queue:
    def __init__(self,leng):
        self.len = leng
        self.que = np.empty(self.len, Request)
        self.front = -1
        self.rear = -1
        
    def display(self):
        if(self.front == -1):
            print("Underflow")
            return
        else:
            i = self.front
            while(i != self.rear):
                print("user:",self.que[i].user)
                i = (i+1)%self.len
            print("user:",self.que[i].user)
            return
    
    def isFull(self):
        return True if((self.rear+1)%self.len == self.front) else False 
                    
    def isEmpty(self):
        return True if(self.front == -1) else False 
    
    def enqueue(self,req):
        if((self.rear+1)%self.len == self.front):
            print("Overflow")
            return False
        if(self.rear == -1):
            self.front += 1
            self.rear += 1
        else:
            self.rear = (self.rear+1)%self.len
        self.que[self.rear] = req
        return True
    
    def dequeue(self):
        if(self.front == -1):
            print("Underflow")
            return False
        req = self.que[self.front]
        if(self.rear == self.front):
            self.front = -1
            self.rear = -1
        else:
            self.front = (self.front+1)%self.len
        return req

In [453]:
class Simulation:
    def __init__(self, num_users, count):
        self.clock = 0.0
        self.event_pq = []
        heapq.heapify(self.event_pq)
        
        self.que = Queue(Q_LENGTH)
        self.s = Server(4)
        
        self.no_of_users = num_users
        self.user = [Users(i, self) for i in range(self.no_of_users)]
        self.req_num = 0
        self.count  = count
            
        self.waitt_list = []
        
        self.req_dropped = 0
        self.req_timedout = 0

    
    def arrival_handler(self,event):
        if not(self.user[event.req.user].put_in_queue(self)):
            return False
        self.s.serve(self, event)
        
    def depart_handler(self,event):
        self.s.serve(self, event)

    def switch_handler(self, event):
        self.s.serve(self, event)
        
    def advance_time(self):
        counter=0
        while(len(self.event_pq) != 0):
            print("---------------Event queue-------------------------")
            for eve in self.event_pq:
                eve.display()
            event = heapq.heappop(self.event_pq)
            self.clock = event.timestamp
            
            if event.e_type == ARR :
                self.arrival_handler(event)
                
            elif event.e_type == DEP :
                counter += 1
                self.depart_handler(event)
                
            elif event.e_type == SWCH:
                self.switch_handler(event)
                
            elif event.e_type == TMT:
                print(f"TMT Event : user {event.req.user}, req {event.req.id}")
                self.user[event.req.user].timeout_handler(event.req.id)
                
            if(counter == self.count):
                break
                
    def display(self):
        print("===================================================")
        print("Clock :",self.clock)
        for i in range(self.no_of_users):
            self.user[i].display()   
        print("---------------Event queue-------------------------")
        for eve in self.event_pq:
            eve.display()
        self.s.display()
        print("---------------Job queue-------------------------")
        self.s.job_que.display()
        print("===================================================")
        

In [454]:
class Request:
    def __init__ (self,req_id, user_id, t_req, tp_timeout, tp_service):
        self.id = req_id
        self.user = user_id
        self.t_req = t_req
        self.tp_timeout = tp_timeout
        self.tp_service = tp_service
        self.tp_wait = 0
        self.core_num = -1
        self.start_wait_t = t_req
        
    def add_to_wait(self,clock):
        self.tp_wait = (clock - self.start_wait_t)


    def assign_core(self, core_num):
        self.core_num = core_num

    def dec_time(self, sw_time):
        self.tp_service = self.tp_service - sw_time

In [455]:
class Users:
    def __init__(self, user_id, sim):
        self.sim = sim
        self.id = user_id
        self.state = THINK
        self.t_req = 0
        self.tp_timeout = 0
        self.tp_service = 0
        self.generate_request(sim)
        
    def display(self):
        print("User",self.id,":-", "State:",self.state, "Request time :",self.t_req, "Service time :",self.tp_service)  

    def generate_thinktime(self):
#         return random.uniform(0,1)
#         return random.triangular(0)
        return np.random.exponential(THINK_TIME)
    
    def generate_service(self):
#         return random.uniform(3,9)
        return np.random.exponential(SERVICE_TIME)
    
    def generate_timeout(self):
        return (TIMEOUT)
    
    def generate_request(self, sim):
        self.t_req = sim.clock + self.generate_thinktime()
        self.tp_timeout = self.generate_timeout()
        self.tp_service = self.generate_service()
        
        sim.req_num += 1
        self.req = Request(sim.req_num, self.id, self.t_req, self.tp_timeout, self.tp_service)
        event = Event(ARR, self.req, self.t_req)
        timeout_event = Event(TMT, self.req, self.t_req + self.tp_timeout)
        heapq.heappush(sim.event_pq, event)
        heapq.heappush(sim.event_pq, timeout_event)
        
    def put_in_queue(self, sim):
        if(not(sim.que.isFull())):
            sim.que.enqueue(self.req)
            self.req.start_wait_t = sim.clock
            self.state = REQ
            self.t_req = float('inf')
            return True
        else:
            self.sim.req_dropped += 1
            self.generate_request(self.sim)
            return False

    def request_finish(self,clock, f_req):
        if(f_req.id == self.req.id):
            self.sim.waitt_list.append(self.req.tp_wait)
            self.state = THINK
            self.generate_request(self.sim)
            
    def timeout_handler(self,req_id):
        if(req_id == self.req.id):
            self.sim.req_timedout += 1
            self.state = THINK
            self.generate_request(self.sim)

In [456]:
class Event:
    def __init__(self, e_type, req, timestamp):
        self.e_type = e_type
        self.req = req
        self.timestamp = timestamp


    def __lt__(self, other):
        return self.timestamp < other.timestamp
    
    def display(self):
        print("Event type",self.e_type,"Timestamp:",self.timestamp,"Req of user:",self.req.user)


In [457]:
class Core:
    def __init__(self, core_id ,sw_time):
        self.core_id = core_id  
        self.state = IDLE
        self.switch_time = sw_time
    def display(self):
        print("Core:",self.core_id,"Core state:",self.state)
        
    def as_request(self, req):
        self.req = req
        self.state = BUSY

    def n_event(self, tstmp):
        if self.switch_time < self.req.tp_service:
            return Event(SWCH, self.req, tstmp + self.switch_time)
        else:
            return Event(DEP, self.req, tstmp + self.req.tp_service)

In [458]:
class Server:
    def __init__(self,cores):
        self.state = IDLE
        self.no_of_cores = cores
        self.cores_list = [Core(i,QUANTUM) for i in range(self.no_of_cores)]
        self.job_que = Queue(15)
        self.n_reqs = 0
        self.max_reqs = 10
    
    def display(self):
        print("No of request in server:", self.n_reqs)
        print("No of cores:", len(self.cores_list))
        print("-----------Cores--------------------")
        for c in self.cores_list:
            c.display()
        
    def serve(self, sim, event):
        if event.e_type == DEP:
            print("------Serving DEP event------")
            sim.user[event.req.user].request_finish(sim.clock, event.req)
            self.n_reqs -= 1
            
            core = self.cores_list[event.req.core_num]
            if not(sim.que.isEmpty()):
                req = sim.que.dequeue()
                self.n_reqs += 1
                req.add_to_wait(sim.clock)
                self.job_que.enqueue(req)
                
            if self.job_que.isEmpty():
                core.state = IDLE
                print(f"Core {core.core_id} is idle.")
                return
            
            ass_req = self.job_que.dequeue()
            core.as_request(ass_req)
            ass_req.assign_core(core.core_id)

            n_event = core.n_event(sim.clock)
            heapq.heappush(sim.event_pq, n_event)
            return

        elif event.e_type == ARR:
            print("---Serving ARR event----")
            if self.n_reqs < self.max_reqs:
                req = sim.que.dequeue()
                self.n_reqs += 1
                req.add_to_wait(sim.clock)
                self.job_que.enqueue(req)
                if self.n_reqs <= self.no_of_cores:
                    for c in self.cores_list:
                        if c.state == IDLE:
                            print(f"Core {c.core_id} found idle.")
                            ass_req = self.job_que.dequeue()
                            c.as_request(ass_req)
                            ass_req.assign_core(c.core_id)

                            n_event = c.n_event(sim.clock)
                            heapq.heappush(sim.event_pq, n_event)
                            break
                else:
                    print("All cores busy.")
            return

        elif event.e_type == SWCH:
#             print("---Serving SWITCH event----")
            core = self.cores_list[event.req.core_num]
            event.req.dec_time(core.switch_time)
            self.job_que.enqueue(event.req)

            ass_req = self.job_que.dequeue()
            core.as_request(ass_req)
            ass_req.assign_core(core.core_id)

            n_event = core.n_event(sim.clock)
            heapq.heappush(sim.event_pq, n_event)
            return


In [459]:
def __main__():
    users = 5
    num_reqs = 10
    sim = Simulation(users, num_reqs)
    sim.advance_time()
    for i in range(sim.no_of_users):
        print(sim.user[i].waitt_list)
    print(len(sim.event_pq))
    for eve in sim.event_pq:
            eve.display()

geb_req()
Event type 1 req 1 of user 0
Event type 8 req 1 of user 0
..finish...
geb_req()
Event type 1 req 2 of user 1
Event type 8 req 2 of user 1
..finish...
geb_req()
Event type 1 req 3 of user 2
Event type 8 req 3 of user 2
..finish...
geb_req()
Event type 1 req 4 of user 3
Event type 8 req 4 of user 3
..finish...
geb_req()
Event type 1 req 5 of user 4
Event type 8 req 5 of user 4
..finish...
---------------Event queue-------------------------
Event type 1 Timestamp: 0.17951630582604203 Req of user: 3
Event type 1 Timestamp: 1.4078042699305773 Req of user: 0
Event type 1 Timestamp: 0.5088746113870388 Req of user: 2
Event type 1 Timestamp: 2.7572464608817935 Req of user: 4
Event type 8 Timestamp: 11.407804269930578 Req of user: 0
Event type 8 Timestamp: 10.50887461138704 Req of user: 2
Event type 1 Timestamp: 3.950237080636348 Req of user: 1
Event type 8 Timestamp: 13.950237080636349 Req of user: 1
Event type 8 Timestamp: 10.179516305826041 Req of user: 3
Event type 8 Timestamp: 12.

Event type 8 Timestamp: 19.011053642234597 Req of user: 1
Event type 8 Timestamp: 12.757246460881793 Req of user: 4
Event type 3 Timestamp: 12.382842349344063 Req of user: 3
TMT Event : user 2, req 3
---------------Event queue-------------------------
Event type 8 Timestamp: 11.249657703042875 Req of user: 2
Event type 8 Timestamp: 11.407804269930578 Req of user: 0
Event type 1 Timestamp: 11.518110533904634 Req of user: 1
Event type 8 Timestamp: 13.950237080636349 Req of user: 1
Event type 3 Timestamp: 12.382842349344063 Req of user: 3
Event type 3 Timestamp: 12.179516305826041 Req of user: 2
Event type 3 Timestamp: 12.332804516748938 Req of user: 0
Event type 8 Timestamp: 21.518110533904633 Req of user: 1
Event type 8 Timestamp: 20.382842349344063 Req of user: 3
Event type 1 Timestamp: 13.692093146812963 Req of user: 4
Event type 8 Timestamp: 23.692093146812965 Req of user: 4
Event type 8 Timestamp: 19.011053642234597 Req of user: 1
Event type 8 Timestamp: 12.757246460881793 Req of us